# Introduction

Sarcasm is a sophisticated language phenomenon, which would cause much confusion to exist sentiment classification systems.     
So sarcasm detection, a task of predicting whether a given text contains sarcasm, has received much research attention.     

Recently, many methods have been proposed for sarcasm detection, which could be broadly classified into two categories.     
One is the text-only method which only concentrate on the utterance itself, such as exploiting incongruity expressions to detect the sarcasm text.     
Another direction is based on extra information, which exploits external knowledge to assist the detection procedure, such as user history, and common sense knowledge.

We propose an unsupervised sarcasm detection method.     

First, we leverage the external sentiment knowledge to mask prominent tokens. Then the masked texts are fed into the pre-trained generation model, which follows the remaining logic structure to generate texts.     
There is a good chance that these reborn texts would not be sarcastic or make more sense.     

Second, after obtaining the similarity score between the generated sentence and the original one, features beneath the scores will be extracted to decide whether a sentence is sarcasm.     

Then, we construct several unsupervised baselines and conduct experiments on IAC-V2 dataset.

# Imports and Reading Data

In [ ]:
import numpy as np
import pandas as pd

from senticnet.senticnet import SenticNet

import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from transformers import AutoTokenizer, AutoModel
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("C:\STUFF 0_0\Deep Learning IITM\Datasets\IAC-V2\sarcasm_v2\GEN-sarc-notsarc.csv")

In [3]:
df= df.drop('id', axis= 1)
df

,class,text
0,notsarc,"If that's true, then Freedom of Speech is doom..."
1,notsarc,Neener neener - is it time to go in from the p...
2,notsarc,"Just like the plastic gun fear, the armour pie..."
3,notsarc,So geology is a religion because we weren't he...
4,notsarc,Well done Monty. Mark that up as your first ev...
...,...,...
6515,sarc,depends on when the baby bird died. run alon...
6516,sarc,"ok, sheesh, to clarify, women who arent aborti..."
6517,sarc,so.. eh?? hows this sound? will it fly w...
6518,sarc,"I think we should put to a vote, the right of ..."


# Understanding Data

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
text_data_original = list(df['text'])
text_data = [x.lower() for x in text_data_original]
print(*text_data, sep = "\n")

In [ ]:
label_data = list(df['class'])
print(*label_data, sep = "\n")

# Overview

The proposed framework contains three main components:     

1) Sentences mask and generation.     
This procedure first recognizes main components of sentences which will be properly masked to cause more impact on original sentences, and then fulfills the texts generation work;     

2) Sentences representation.     
It is expected to calculate dense vectors of sentences;     

3) Sarcastic utterances detection leverages.     
the similarity scores between original and regenerated sentences to detect whether an utterance is sarcastic.

# Sentences Mask and Generation
## 1)
"First, we use the sentiment common knowledge retrieved from SenticNet to recognize affective words in the sentence 𝑥,     
and split those words into two sets according to its sentiment polarities:    
PW = {pw1, pw2, ..., pwh} and    
NW = {nw1, nw2, ..., nwk},     
h + k <= n."

In [8]:
def tokenize_sentence(sentence):
    tokens = word_tokenize(sentence)
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [9]:
def get_sentiment_polarity_from_senticnet(word):
    sn = SenticNet()

    word = word.lower()

    try:
        return sn.polarity_label(word)
    except:
        return "neutral"

In [10]:
def analyze_sentiment(sentences):
    positive_words = []
    negative_words = []

    for sentence in sentences:
        words = tokenize_sentence(sentence)

        PW = set()
        NW = set()

        for word in words:
            sentiment_polarity = get_sentiment_polarity_from_senticnet(word)
            if sentiment_polarity == "positive":
                PW.add(word.lower())
            elif sentiment_polarity == "negative":
                NW.add(word.lower())

        positive_words.append(PW)
        negative_words.append(NW)

    return positive_words, negative_words

In [ ]:
positive_words, negative_words = analyze_sentiment(text_data)

for i, sentence in enumerate(text_data):
    print(f"Sentence: {sentence}")
    print(f"Positive Words: {positive_words[i]}")
    print(f"Negative Words: {negative_words[i]}")
    print("- - - - - - - - - -")

In [12]:
df["PW"] = positive_words
df["NW"] = negative_words
df

,class,text,PW,NW
0,notsarc,"If that's true, then Freedom of Speech is doom...","{true, freedom, subjective}","{doomed, harassment}"
1,notsarc,Neener neener - is it time to go in from the p...,{playground},{}
2,notsarc,"Just like the plastic gun fear, the armour pie...",{},"{myth, plastic, fear, misinformation}"
3,notsarc,So geology is a religion because we weren't he...,"{formed, rock, religion}",{geology}
4,notsarc,Well done Monty. Mark that up as your first ev...,"{honest, accurate}",{}
...,...,...,...,...
6515,sarc,depends on when the baby bird died. run alon...,"{baby, bird, adult}",{died}
6516,sarc,"ok, sheesh, to clarify, women who arent aborti...","{baby, ok, clarify}","{pregnant, sheesh}"
6517,sarc,so.. eh?? hows this sound? will it fly w...,"{fly, progressive, conservative}",{}
6518,sarc,"I think we should put to a vote, the right of ...",{majority},{extremist}


## 2)
"Second, we analyze the sentence to get its syntax information to identify non-stop words     
     𝑆𝑊 = {𝑠𝑤1, 𝑠𝑤2, ..., 𝑠𝑤𝑚, 𝑚 ≤ 𝑛}.     
Intuitively, these words are the main components of sentences. Then we split 𝑆𝑊 into two sets which satisfy :     
     𝑆𝑊1 ∪ 𝑆𝑊2 = 𝑆𝑊 ,     
     |𝑆𝑊1| = |𝑆𝑊2|."

In [13]:
def extract_non_stop_words(sentence):
    words = nltk.word_tokenize(sentence)

    stop_words = set(stopwords.words("english"))

    non_stop_words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]

    return non_stop_words

In [14]:
def split_non_stop_words(non_stop_words):
    m = len(non_stop_words)
    m1 = m // 2
    SW1 = set(non_stop_words[:m1])
    SW2 = set(non_stop_words[m1:])
    return SW1, SW2

In [15]:
def analyze_sentences(sentences):
    all_SW1 = []
    all_SW2 = []

    for sentence in sentences:
        non_stop_words = extract_non_stop_words(sentence)
        SW1, SW2 = split_non_stop_words(non_stop_words)
        all_SW1.append(SW1)
        all_SW2.append(SW2)

    return all_SW1, all_SW2

In [ ]:
all_SW1, all_SW2 = analyze_sentences(text_data)

for i, sentence in enumerate(text_data):
    print(f"Sentence: {sentence}")
    print(f"SW1: {all_SW1[i]}")
    print(f"SW2: {all_SW2[i]}")
    print("- - - - - - - - - -")

In [17]:
df["SW1"] = all_SW1
df["SW2"] = all_SW2
df

,class,text,PW,NW,SW1,SW2
0,notsarc,"If that's true, then Freedom of Speech is doom...","{true, freedom, subjective}","{doomed, harassment}","{harassment, freedom, true, doomed, speech}","{like, claim, book, subjective, harassing, ban..."
1,notsarc,Neener neener - is it time to go in from the p...,{playground},{},"{neener, time}","{playground, go, yet}"
2,notsarc,"Just like the plastic gun fear, the armour pie...",{},"{myth, plastic, fear, misinformation}","{like, plastic, gun, armour, fear, piercing}","{myth, misinformation, bullet, upon, built, fear}"
3,notsarc,So geology is a religion because we weren't he...,"{formed, rock, religion}",{geology},"{see, religion, geology}","{x, rock, formed}"
4,notsarc,Well done Monty. Mark that up as your first ev...,"{honest, accurate}",{},"{well, mark, done, monty}","{first, post, honest, accurate, ever}"
...,...,...,...,...,...,...
6515,sarc,depends on when the baby bird died. run alon...,"{baby, bird, adult}",{died},"{died, run, bird, baby, depends}","{let, adults, boy, along, debate, little}"
6516,sarc,"ok, sheesh, to clarify, women who arent aborti...","{baby, ok, clarify}","{pregnant, sheesh}","{babys, women, clarify, ok, aborting, arent, s...","{times, women, pregnant, abortons, year, sever..."
6517,sarc,so.. eh?? hows this sound? will it fly w...,"{fly, progressive, conservative}",{},"{fly, sound, eh, hows}","{conservatives, thezion, progressives, think}"
6518,sarc,"I think we should put to a vote, the right of ...",{majority},{extremist},"{religious, vote, extremists, put, right, ye, ...","{anymore, steeeeeve, says, vote, done, put, ag..."


## 3)
"Here, 𝑃𝑊 ∪ 𝑆𝑊1 and 𝑁𝑊 ∪ 𝑆𝑊2 are used to mask original sentence respectively. So, we will obtain two masked sentences     
𝑥𝑚1 = { [𝑚]1, 𝑥2, ..., [𝑚]𝑛} and     
𝑥𝑚2 = {𝑥1, [𝑚]2, ..., 𝑥𝑛}."

In [18]:
def construct_union(sentences, PW, NW, all_SW1, all_SW2):
    union_PW_SW1 = []
    union_NW_SW2 = []

    for i, sentence in enumerate(sentences):
        SW1 = all_SW1[i]
        SW2 = all_SW2[i]
        
        union_PW_SW1.append(PW[i].union(SW1))
        union_NW_SW2.append(NW[i].union(SW2))

    return union_PW_SW1, union_NW_SW2

In [ ]:
union_PW_SW1, union_NW_SW2 = construct_union(text_data, positive_words, negative_words, all_SW1, all_SW2)
print(union_PW_SW1)
print(union_NW_SW2)

In [20]:
df["union_PW_SW1"] = union_PW_SW1
df["union_NW_SW2"] = union_NW_SW2
df

,class,text,PW,NW,SW1,SW2,union_PW_SW1,union_NW_SW2
0,notsarc,"If that's true, then Freedom of Speech is doom...","{true, freedom, subjective}","{doomed, harassment}","{harassment, freedom, true, doomed, speech}","{like, claim, book, subjective, harassing, ban...","{harassment, subjective, freedom, true, doomed...","{harassment, like, claim, book, subjective, do..."
1,notsarc,Neener neener - is it time to go in from the p...,{playground},{},"{neener, time}","{playground, go, yet}","{playground, neener, time}","{playground, go, yet}"
2,notsarc,"Just like the plastic gun fear, the armour pie...",{},"{myth, plastic, fear, misinformation}","{like, plastic, gun, armour, fear, piercing}","{myth, misinformation, bullet, upon, built, fear}","{like, fear, plastic, gun, piercing, armour}","{myth, misinformation, bullet, plastic, upon, ..."
3,notsarc,So geology is a religion because we weren't he...,"{formed, rock, religion}",{geology},"{see, religion, geology}","{x, rock, formed}","{rock, formed, see, religion, geology}","{x, rock, formed, geology}"
4,notsarc,Well done Monty. Mark that up as your first ev...,"{honest, accurate}",{},"{well, mark, done, monty}","{first, post, honest, accurate, ever}","{honest, done, monty, well, mark, accurate}","{honest, ever, accurate, first, post}"
...,...,...,...,...,...,...,...,...
6515,sarc,depends on when the baby bird died. run alon...,"{baby, bird, adult}",{died},"{died, run, bird, baby, depends}","{let, adults, boy, along, debate, little}","{died, run, bird, adult, baby, depends}","{let, died, adults, boy, along, debate, little}"
6516,sarc,"ok, sheesh, to clarify, women who arent aborti...","{baby, ok, clarify}","{pregnant, sheesh}","{babys, women, clarify, ok, aborting, arent, s...","{times, women, pregnant, abortons, year, sever...","{babys, women, clarify, ok, aborting, baby, ar...","{times, women, pregnant, abortons, sheesh, yea..."
6517,sarc,so.. eh?? hows this sound? will it fly w...,"{fly, progressive, conservative}",{},"{fly, sound, eh, hows}","{conservatives, thezion, progressives, think}","{fly, sound, progressive, hows, eh, conservative}","{conservatives, thezion, progressives, think}"
6518,sarc,"I think we should put to a vote, the right of ...",{majority},{extremist},"{religious, vote, extremists, put, right, ye, ...","{anymore, steeeeeve, says, vote, done, put, ag...","{religious, say, vote, extremists, put, right,...","{anymore, steeeeeve, says, extremist, vote, do..."


In [21]:
def mask_sentence(sentence, mask_words, max_mask_count = 5):
    masked_sentence = []
    
    for word in sentence.split():
        if word in mask_words and max_mask_count > 0:
            masked_sentence.append("<mask>")
            max_mask_count -= 1
        else:
            masked_sentence.append(word)
    
    return " ".join(masked_sentence)

In [22]:
def construct_masked_sentences(sentences, union_PW_SW1, union_NW_SW2):
    masked_pos_sentences = []
    masked_neg_sentences = []

    for i, sentence in enumerate(sentences):

        masked_pos_sentence = mask_sentence(sentence, union_PW_SW1[i])
        masked_pos_sentences.append(masked_pos_sentence)

        masked_neg_sentence = mask_sentence(sentence, union_NW_SW2[i])
        masked_neg_sentences.append(masked_neg_sentence)

    return masked_pos_sentences, masked_neg_sentences

In [ ]:
masked_pos_sentences, masked_neg_sentences = construct_masked_sentences(text_data, union_PW_SW1, union_NW_SW2)

for i, sentence in enumerate(text_data):
    print(f"Original Sentence: {sentence}")
    print(f"Masked Positive Sentence: {masked_pos_sentences[i]}")
    print(f"Masked Negative Sentence: {masked_neg_sentences[i]}")
    print("- - - - - - - - - -")

In [24]:
dfnew = pd.DataFrame({"text": text_data_original, "maskedPosSentence": masked_pos_sentences, "maskedNegSentence": masked_neg_sentences})
dfnew

,text,maskedPosSentence,maskedNegSentence
0,"If that's true, then Freedom of Speech is doom...","if that's true, then <mask> of <mask> is doome...","if that's true, then freedom of speech is doom..."
1,Neener neener - is it time to go in from the p...,<mask> <mask> - is it <mask> to go in from the...,neener neener - is it time to <mask> in from t...
2,"Just like the plastic gun fear, the armour pie...","just <mask> the <mask> <mask> fear, the <mask>...","just like the <mask> gun fear, the armour pier..."
3,So geology is a religion because we weren't he...,so <mask> is a <mask> because we weren't here ...,so <mask> is a religion because we weren't her...
4,Well done Monty. Mark that up as your first ev...,<mask> <mask> monty. <mask> that up as your fi...,well done monty. mark that up as your <mask> <...
...,...,...,...
6515,depends on when the baby bird died. run alon...,<mask> on when the <mask> <mask> died. <mask> ...,depends on when the baby bird died. run <mask>...
6516,"ok, sheesh, to clarify, women who arent aborti...","ok, sheesh, to clarify, <mask> who <mask> <mas...","ok, sheesh, to clarify, <mask> who arent abort..."
6517,so.. eh?? hows this sound? will it fly w...,so.. eh?? <mask> this sound? will it <mask> wi...,so.. eh?? hows this sound? will it fly with <m...
6518,"I think we should put to a vote, the right of ...","i <mask> we should <mask> to a vote, the <mask...","i think we should <mask> to a vote, the right ..."


## 4)
"These two masked sentences are fed into the pre-trained generation model to fulfill the generation procedure.     
𝑨{𝑎1, ..., 𝑥2, ..., 𝑥𝑛−1, ..., 𝑎𝑜 } = 𝐵𝐴𝑅𝑇 ( [𝑚]1, 𝑥2, ..., 𝑥𝑛−1, [𝑚]𝑛 )----(1)  
Thus, we will obtain two reborn sentences     
𝐴 = {𝑎1, 𝑎2, ..., 𝑎𝑜 } and     
𝐵 = {𝑏1, 𝑏2, ..., 𝑏𝑝 }."

In [ ]:
%pip install transformers

In [26]:
def generate_reborn_sentences(masked_sentences):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

    i = 0
    reborn_sentences = []
    for masked_sentence in masked_sentences:
        inputs = tokenizer(masked_sentence, return_tensors="pt")
        generated_encoded = model.generate(inputs['input_ids'])
        reborn_sentence = tokenizer.batch_decode(generated_encoded, skip_special_tokens=True)[0]
        reborn_sentences.append(reborn_sentence)
        i = i + 1
        if (i % 100 == 0):
            print(f'Processed {i} sentences')
    
    return reborn_sentences

In [ ]:
reborn_pos_sentences = generate_reborn_sentences(masked_pos_sentences)

reborn_neg_sentences = generate_reborn_sentences(masked_neg_sentences)

In [ ]:
print("Reborn Sentences for Masked Positive Sentences:")
for i, reborn_sentence in enumerate(reborn_pos_sentences):
    print(f"Reborn Sentence {i + 1}: {reborn_sentence}")

In [ ]:
print("\nReborn Sentences for Masked Negative Sentences:")
for i, reborn_sentence in enumerate(reborn_neg_sentences):
    print(f"Reborn Sentence {i + 1}: {reborn_sentence}")

In [30]:
dfnew["rebornPosSentence"] = reborn_pos_sentences
dfnew["rebornNegSentence"] = reborn_neg_sentences
dfnew

,text,maskedPosSentence,maskedNegSentence,rebornPosSentence,rebornNegSentence
0,"If that's true, then Freedom of Speech is doom...","if that's true, then <mask> of <mask> is doome...","if that's true, then freedom of speech is doom...","if that's true, then the whole idea of banning...","if that's true, then freedom of speech is doom..."
1,Neener neener - is it time to go in from the p...,<mask> <mask> - is it <mask> to go in from the...,neener neener - is it time to <mask> in from t...,So - is it time to go in from the airport yet?,neener neener - is it time to step in from the...
2,"Just like the plastic gun fear, the armour pie...","just <mask> the <mask> <mask> fear, the <mask>...","just like the <mask> gun fear, the armour pier...","just like the first bullet fear, the second bu...","just like the gun fear, the armour piercing fe..."
3,So geology is a religion because we weren't he...,so <mask> is a <mask> because we weren't here ...,so <mask> is a religion because we weren't her...,so this is a joke because we weren't here to s...,so what is a religion because we weren't here ...
4,Well done Monty. Mark that up as your first ev...,<mask> <mask> monty. <mask> that up as your fi...,well done monty. mark that up as your <mask> <...,This is a monty. You can pick that up as your ...,well done monty. mark that up as your own. 100...
...,...,...,...,...,...
6515,depends on when the baby bird died. run alon...,<mask> on when the <mask> <mask> died. <mask> ...,depends on when the baby bird died. run <mask>...,No word on when the boy died. All along little...,depends on when the baby bird died. run away f...
6516,"ok, sheesh, to clarify, women who arent aborti...","ok, sheesh, to clarify, <mask> who <mask> <mas...","ok, sheesh, to clarify, <mask> who arent abort...","ok, sheesh, to clarify, the women who are havi...","ok, sheesh, to clarify, those who arent aborti..."
6517,so.. eh?? hows this sound? will it fly w...,so.. eh?? <mask> this sound? will it <mask> wi...,so.. eh?? hows this sound? will it fly with <m...,so.. eh?? how does this sound? will it resonat...,so.. eh?? hows this sound? will it fly with co...
6518,"I think we should put to a vote, the right of ...","i <mask> we should <mask> to a vote, the <mask...","i think we should <mask> to a vote, the right ...","i don't think we should put it to a vote, the ...","i think we should put it to a vote, the right ..."


# Sentences Representation
"We embed the original sentence 𝑥 and its corresponding reborn texts 𝐴 and 𝐵     
into 𝑑-dimentional embedding 𝑯𝑡 ∈ R𝑑     
via pre-trained BERT-base:     
𝑯𝑥, 𝑯𝐴, 𝑯𝐵 = 𝐵𝐸𝑅𝑇 (𝑥), 𝐵𝐸𝑅𝑇 (𝐴), 𝐵𝐸𝑅𝑇 (𝐵)."

In [31]:
def embed_sentences(sentences):
    tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
    model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

    i = 0
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs).last_hidden_state.mean(dim=1)
        embeddings.append(outputs)
        i = i + 1
        if (i % 100 == 0):
            print(f'Processed {i} sentences')

    return torch.stack(embeddings)

In [ ]:
x_embeddings = embed_sentences(text_data)

A_embeddings = embed_sentences(reborn_pos_sentences)

B_embeddings = embed_sentences(reborn_neg_sentences)

In [ ]:
for i, sentence in enumerate(text_data):
    print(f"Embedding for Original Lowercase Sentence {i + 1} ({sentence}):")
    print(x_embeddings[i])
    print("- - - - - - - - - -")

In [ ]:
for i, sentence in enumerate(reborn_pos_sentences):
    print(f"Embedding for Reborn Positive Sentence {i + 1} ({sentence}):")
    print(A_embeddings[i])
    print("- - - - - - - - - -")

In [ ]:
for i, sentence in enumerate(reborn_neg_sentences):
    print(f"Embedding for Reborn Negative Sentence {i + 1} ({sentence}):")
    print(B_embeddings[i])
    print("- - - - - - - - - -")

In [36]:
dfnew["xEmbedding"] = x_embeddings.tolist()
dfnew["AEmbedding"] = A_embeddings.tolist()
dfnew["BEmbedding"] = B_embeddings.tolist()
dfnew

,text,maskedPosSentence,maskedNegSentence,rebornPosSentence,rebornNegSentence,xEmbedding,AEmbedding,BEmbedding
0,"If that's true, then Freedom of Speech is doom...","if that's true, then <mask> of <mask> is doome...","if that's true, then freedom of speech is doom...","if that's true, then the whole idea of banning...","if that's true, then freedom of speech is doom...","[[0.2883792221546173, 0.4140245020389557, 0.54...","[[0.40408700704574585, 0.4816318452358246, 0.1...","[[0.353076696395874, 0.35502904653549194, 0.21..."
1,Neener neener - is it time to go in from the p...,<mask> <mask> - is it <mask> to go in from the...,neener neener - is it time to <mask> in from t...,So - is it time to go in from the airport yet?,neener neener - is it time to step in from the...,"[[0.5522845387458801, -0.7339646816253662, 0.0...","[[0.38219159841537476, -0.6355235576629639, 0....","[[0.36270591616630554, -0.5871939063072205, -0..."
2,"Just like the plastic gun fear, the armour pie...","just <mask> the <mask> <mask> fear, the <mask>...","just like the <mask> gun fear, the armour pier...","just like the first bullet fear, the second bu...","just like the gun fear, the armour piercing fe...","[[0.6633205413818359, 0.47166547179222107, -0....","[[0.4542856216430664, 0.25347062945365906, -0....","[[0.616788387298584, 0.31699368357658386, -0.1..."
3,So geology is a religion because we weren't he...,so <mask> is a <mask> because we weren't here ...,so <mask> is a religion because we weren't her...,so this is a joke because we weren't here to s...,so what is a religion because we weren't here ...,"[[0.6581799983978271, 0.482073575258255, -0.14...","[[0.7056753039360046, 0.038419123739004135, -0...","[[0.5304417014122009, 0.20461539924144745, -0...."
4,Well done Monty. Mark that up as your first ev...,<mask> <mask> monty. <mask> that up as your fi...,well done monty. mark that up as your <mask> <...,This is a monty. You can pick that up as your ...,well done monty. mark that up as your own. 100...,"[[0.15030699968338013, 0.34413737058639526, 0....","[[-0.07571116089820862, 0.027956565842032433, ...","[[0.22033759951591492, 0.5279558300971985, 0.4..."
...,...,...,...,...,...,...,...,...
6515,depends on when the baby bird died. run alon...,<mask> on when the <mask> <mask> died. <mask> ...,depends on when the baby bird died. run <mask>...,No word on when the boy died. All along little...,depends on when the baby bird died. run away f...,"[[0.25484198331832886, 0.08131378889083862, -0...","[[0.0312921516597271, 0.290886253118515, 0.401...","[[0.2597534656524658, -0.2652691900730133, -0...."
6516,"ok, sheesh, to clarify, women who arent aborti...","ok, sheesh, to clarify, <mask> who <mask> <mas...","ok, sheesh, to clarify, <mask> who arent abort...","ok, sheesh, to clarify, the women who are havi...","ok, sheesh, to clarify, those who arent aborti...","[[0.06550610065460205, 0.1687377244234085, -0....","[[0.22560596466064453, 0.35204559564590454, -0...","[[0.5040947198867798, 0.13611070811748505, 0.0..."
6517,so.. eh?? hows this sound? will it fly w...,so.. eh?? <mask> this sound? will it <mask> wi...,so.. eh?? hows this sound? will it fly with <m...,so.. eh?? how does this sound? will it resonat...,so.. eh?? hows this sound? will it fly with co...,"[[0.2706380784511566, 0.1423533409833908, 0.21...","[[0.1413293331861496, -0.2442535161972046, 0.4...","[[0.22468434274196625, -0.27893364429473877, 0..."
6518,"I think we should put to a vote, the right of ...","i <mask> we should <mask> to a vote, the <mask...","i think we should <mask> to a vote, the right ...","i don't think we should put it to a vote, the ...","i think we should put it to a vote, the right ...","[[0.24415284395217896, -0.06654065102338791, 0...","[[0.32860881090164185, -0.5744808316230774, 0....","[[0.12385067343711853, -0.08365674316883087, 0..."


# Sarcastic Utterances Detection
## 1)
"We utilize cosine similarity to measure the similarity between representations of original sentence 𝐻𝑥     
and generation texts 𝐻𝐴/𝐻𝐵.

Then we use the following equation to calculate a difference score of each sentence:     
diff = sim(𝐻𝑥, 𝐻𝐴) < 𝑡ℎ𝑟𝑒𝑠ℎ𝑜𝑙𝑑 || sim(𝐻𝑥, 𝐻𝐵) < 𝑡ℎ𝑟𝑒𝑠ℎ𝑜𝑙𝑑     
where || means "or" logical operator."

In [37]:
def calculate_difference_scores(x_embeddings, A_embeddings, B_embeddings, threshold):
    i = 0
    diff_scores = []
    for x_emb, A_emb, B_emb in zip(x_embeddings, A_embeddings, B_embeddings):
        sim_Hx_HA = cosine_similarity(x_emb, A_emb)
        sim_Hx_HB = cosine_similarity(x_emb, B_emb)
        
        diff = (sim_Hx_HA < threshold) or (sim_Hx_HB < threshold)
        diff_scores.append(diff)
        i = i + 1
        if (i % 100 == 0):
            print(f'Processed {i} embeddings')

    return diff_scores

In [ ]:
threshold = 0.755

diff_scores = calculate_difference_scores(x_embeddings, A_embeddings, B_embeddings, threshold)
diff_scores

## 2)
"Since the sarcastic utterances are influenced more than normal texts during the masking and generation procedure,     
the difference score of sarcastic texts should be greater than a non-sarcastic one.

If we have a threshold value which separates sarcastic texts and normal texts,     
we can yield the prediction 𝑦 by:     
𝑦 = I(diff)."

In [ ]:
predicted_labels = [int(diff) for diff in diff_scores]
print(predicted_labels)
print(sum(predicted_labels))

In [ ]:
labels = ["sarc" if diff else "notsarc" for diff in diff_scores]
print(labels)

In [ ]:
dffinal = pd.DataFrame({"text": text_data, "class": label_data, "prediction": labels})
dffinal

# Main Experiment Results

In [42]:
true_labels = [1 if pred == "sarc" else 0 for pred in df["class"]]
print(true_labels)
print(predicted_labels)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [43]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[1550 1710]
 [1607 1653]]
